# Hackathon : 카드 연체기간 예측 및 경향분석
---

## [Issues]
1. ID, Name and SSN (Not useful)
2. Age, Annual_Income, Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Amount_invested_monthly, Outstanding_Debt Credit_Mix, Monthly_Balance Numerical but show as catogery (need to be fixed)
3. Occupation, CreditMix has value "__"
4. Data contains outliers
5. Num_Credit_Card has zeros
6. Type_of_Loan Need to rewrite as 8 columns
7. Num_Bank_Accounts contains negative values
8. Credit_History_Age,Payment_of_Min_Amount,Payment_Behaviour,'Credit_Mix' (needs Feature Engineering)
9. Target Columns is Imbalanced
10. A lot of missing data


# 1. 라이브러리,데이터 import

In [7]:
# Packages for EDA 
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd 
import numpy as np 

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from datasist.structdata import detect_outliers
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import category_encoders as ce
import re 

# Modeling and evaluation 
from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from sklearn.metrics import classification_report 
# import joblib # 파이프제작 파이프라인

# Packages options 
sns.set(rc={'figure.figsize': [14, 7]}, font_scale=1.2) # Standard figure size for all 
np.seterr(divide='ignore', invalid='ignore', over='ignore') ;

import warnings 
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_csv('./csv/train.csv')
df.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good


# 2. Preprocessing

결측치 확인 및 imputation

In [11]:
# X.head(2)
df.info() # 여기서 object를 int, float으로 바꿔야함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Age                       100000 non-null  object 
 3   Occupation                100000 non-null  object 
 4   Annual_Income             100000 non-null  object 
 5   Monthly_Inhand_Salary     84998 non-null   float64
 6   Num_Bank_Accounts         100000 non-null  int64  
 7   Num_Credit_Card           100000 non-null  int64  
 8   Interest_Rate             100000 non-null  int64  
 9   Num_of_Loan               100000 non-null  object 
 10  Type_of_Loan              88592 non-null   object 
 11  Delay_from_due_date       100000 non-null  int64  
 12  Num_of_Delayed_Payment    92998 non-null   object 
 13  Changed_Credit_Limit      100000 non-null  ob

## 1 ) 중복컬럼 제외

In [10]:
# df.columns
df = df.drop(["ID", "Name", "SSN"], axis=1) # 종속변인+중복컬럼 제외

# X = StandardScaler().fit_transform(X) <- 정규화는 나중에! 중요도평가할 때 음수값 있으면 안됌

## 2 ) 언더바"_" 있는 값 처리

In [12]:
N_to_fix = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment',
            'Changed_Credit_Limit', 'Amount_invested_monthly', 'Outstanding_Debt' , 'Monthly_Balance']

def fix_nums(num):
    try : 
        return float(num.replace("_",""))
    except :
        return np.nan
    
for col in N_to_fix :
    df[col] = df[col].apply(fix_nums)

## 3 ) 대출유형 컬럼나누기
- 모든 대출유형은 서로 다른 9개의 대출의 조합으로 이루어진다.
- 그래서 컬럼 9개를 만들어 각 행데이터마다 해당 대출을 갖고있는지에 대해 false(안가짐), true(가짐) 처리.
- 위 수행 후 기존 대출유형 컬럼 삭제

In [13]:
# X["Type_of_Loan"].value_counts().sum()
df["Type_of_Loan"].value_counts().head(9)

Not Specified              1408
Credit-Builder Loan        1280
Personal Loan              1272
Debt Consolidation Loan    1264
Student Loan               1240
Payday Loan                1200
Mortgage Loan              1176
Auto Loan                  1152
Home Equity Loan           1136
Name: Type_of_Loan, dtype: int64

In [14]:
for i in df['Type_of_Loan'].value_counts().head(9).index[1:] : 
    df[i] = df['Type_of_Loan'].str.contains(i)

del df['Type_of_Loan']

## 4 ) 마이너스 값 처리 + 0값 대체(카드,계좌수)

In [15]:
df.describe()
# min값에 Num_Bank_Accounts만 음수임. 음수 존재.
# Num_Credit_Card가 0 일 수 없음 -> 최소 1개이니 0을 1로 대체

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,100000.000000,1.000000e+05,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,92998.000000,97909.000000,98035.000000,100000.000000,100000.000000,100000.000000,95521.000000,9.713200e+04
mean,110.649700,1.764157e+05,4194.170850,17.091280,22.47443,72.466040,3.009960,21.068780,30.923342,10.389025,27.754251,1426.220376,32.285173,1403.118217,637.412998,-3.088580e+22
std,686.244717,1.429618e+06,3183.686167,117.404834,129.05741,466.422621,62.647879,14.860104,226.031892,6.789496,193.177339,1155.129026,5.116875,8306.041270,2043.319327,3.208492e+24
min,-500.000000,7.005930e+03,303.645417,-1.000000,0.00000,1.000000,-100.000000,-5.000000,-3.000000,-6.490000,0.000000,0.230000,20.000000,0.000000,0.000000,-3.333333e+26
25%,24.000000,1.945750e+04,1625.568229,3.000000,4.00000,8.000000,1.000000,10.000000,9.000000,5.320000,3.000000,566.072500,28.052567,30.306660,74.534002,2.699858e+02
50%,33.000000,3.757861e+04,3093.745000,6.000000,5.00000,13.000000,3.000000,18.000000,14.000000,9.400000,6.000000,1166.155000,32.305784,69.249473,135.925682,3.364493e+02
75%,42.000000,7.279092e+04,5957.448333,7.000000,7.00000,20.000000,5.000000,28.000000,18.000000,14.870000,9.000000,1945.962500,36.496663,161.224249,265.731733,4.696405e+02
max,8698.000000,2.419806e+07,15204.633333,1798.000000,1499.00000,5797.000000,1496.000000,67.000000,4397.000000,36.970000,2597.000000,4998.070000,50.000000,82331.000000,10000.000000,1.602041e+03


In [16]:
df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x :abs (x)) #절댓값으로 변환
df['Num_Bank_Accounts'].describe()

count    100000.000000
mean         17.091700
std         117.404773
min           0.000000
25%           3.000000
50%           6.000000
75%           7.000000
max        1798.000000
Name: Num_Bank_Accounts, dtype: float64

In [ ]:
# 은행, 카드 갯수 0을 1개로
df['Num_Bank_Accounts'].replace(0,1,inplace=True)
df['Num_Credit_Card'].replace(0,1,inplace=True)

## 5 ) 결측치 처리

년을 월로 계산 : 'Credit_History_Age' 컬럼

In [12]:
def History_age(age):
    try : 
        years = int("".join(re.findall('[0-9]',''.join(age.split("and")[0]))))
        month = int("".join(re.findall('[0-9]',''.join(age.split("and")[1]))))
        return years*12 + month
    except :
        return np.nan
    
df['Credit_History_Age'] = df['Credit_History_Age'].apply(History_age)


### 결측치 대체(replace)

"Payment_of_Min_Amount" 결측치 처리

In [13]:
df['Payment_of_Min_Amount'].value_counts()

Yes    52326
No     35667
NM     12007
Name: Payment_of_Min_Amount, dtype: int64

In [14]:
df['Payment_of_Min_Amount'].replace("NM","No",inplace=True)
df['Payment_of_Min_Amount'].value_counts()

Yes    52326
No     47674
Name: Payment_of_Min_Amount, dtype: int64

"Payment_Behaviour" 결측치 처리

In [15]:
df['Payment_Behaviour'].value_counts()
# 결측치 존재 -> "!@9#%8"

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: Payment_Behaviour, dtype: int64

In [16]:
df['Payment_Behaviour']= df['Payment_Behaviour'].replace("!@9#%8",np.nan) # 결측치 처리
df['Payment_Behaviour'].value_counts()

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
Name: Payment_Behaviour, dtype: int64

### 결측치 대체 : 같은사람(customer id) 데이터 중 결측치가 없는 데이터로 대체

"Occupation" 결측치 처리 : "_______"

In [17]:
df['Occupation'].value_counts() # _______ 값 존재.

_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64

결측치 대체 : 같은사람(customer id) 데이터 중 결측치가 없는 데이터로 대체

In [18]:
occs = df['Occupation'].value_counts().index[1:]
occs

Index(['Lawyer', 'Architect', 'Engineer', 'Scientist', 'Mechanic',
       'Accountant', 'Developer', 'Media_Manager', 'Teacher', 'Entrepreneur',
       'Doctor', 'Journalist', 'Manager', 'Musician', 'Writer'],
      dtype='object')

In [19]:
# 직업이 _____로 되어있는 칸이 있어서 같은 사람의 직업 데이터로 대체시켜준다.
for ID in df[df['Occupation'] == "_______"]['Customer_ID'] : 
    oc = df[df['Customer_ID'] == ID]['Occupation'].mode()[0]
    df[df['Customer_ID'] == ID] = df[df['Customer_ID'] == ID].replace("_______",oc)

In [20]:
df['Occupation'].value_counts()
# 대체 시켜줄 데이터가 없는 행이 5개 존재하기에 아래 코드 진행.

Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6715
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
_______             5
Name: Occupation, dtype: int64

In [21]:
df['Occupation'] = df['Occupation'].replace("_______",df['Occupation'].mode()[0])
df['Occupation'].value_counts()
# "______" 대체, 제거 완료.

Lawyer           7101
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6715
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: Occupation, dtype: int64

'Credit_Mix' 결측치 처리 : 대체 + 카테고리화

In [22]:
df['Credit_Mix'].value_counts() # _ 존재. np로 처리.

Standard    36479
Good        24337
_           20195
Bad         18989
Name: Credit_Mix, dtype: int64

In [23]:
m = {
    "Bad":0,
    "Standard":1,
    "Good":2,
    "_":np.nan
}

df['Credit_Mix'] = df['Credit_Mix'].map(m)

In [24]:
df['Credit_Mix'].value_counts()

1.0    36479
2.0    24337
0.0    18989
Name: Credit_Mix, dtype: int64

### 이상치 처리
기준치를 넘는 데이터는 np로 처리하여 이후 Imputer를 통해 같은 사람의 결측치가 없는 데이터로 대체될 수 있도록 한다.

In [ ]:
import numpy as np
for i in range(len(df["Num_of_Loan"])):
    if df["Num_of_Loan"][i] >= 10 :
        df["Num_of_Loan"][i] = np.nan
    else:
        pass

for i in range(len(df["Interest_Rate"])):
    if df["Interest_Rate"][i] >= 150 :
        df["Interest_Rate"][i] = np.nan
    else:
        pass

for i in range(len(df["Num_Bank_Accounts"])):
    if df["Num_Bank_Accounts"][i] >= 20 :
        df["Num_Bank_Accounts"][i] = np.nan
    else:
        pass

for i in range(len(df["Num_Credit_Card"])):
    if df["Num_Credit_Card"][i] >= 20 :
        df["Num_Credit_Card"][i] = np.nan
    else:
        pass

### *이상치 및 결측치 대체(KNNImputer)
Missing Data를 같은 사람의 다른 데이터 평균 값으로 대체해주기(수치형, 스트링형 따로)

In [25]:
# Edit Columns from bool to int 
for col in list(df.columns[-8:]):
    df[col] = df[col].astype(float)
    
# 각 고객ID를 숫자(1~)로 변경
IDs = 1 
for ID in df['Customer_ID'].unique() :
    df['Customer_ID'] = df['Customer_ID'].replace(ID,IDs)
    IDs += 1

In [168]:
# KNNImputer를 사용하여 결측치를 근처에 있는 값의 평균으로 대체.(같은 사람 데이터 중)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)

## 먼저 수치형 데이터 먼저
Numericals = df.select_dtypes(exclude='object').columns[1:]
Numericals

Index(['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance', 'Credit-Builder Loan',
       'Personal Loan', 'Debt Consolidation Loan', 'Student Loan',
       'Payday Loan', 'Mortgage Loan', 'Auto Loan', 'Home Equity Loan'],
      dtype='object')

> imputer code. 시간소요 큼

In [169]:
for col in Numericals[1:]:
    imputer.fit(df[['Customer_ID',col]])
    df[['Customer_ID',col]] = imputer.transform(df[['Customer_ID',col]])

df.info()

# Customer_ID 이용해서 다른 값들 대체했기에 이제 쓸모가 없어서 열 삭제!
del df['Customer_ID']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  float64
 1   Month                     100000 non-null  object 
 2   Age                       100000 non-null  float64
 3   Occupation                100000 non-null  object 
 4   Annual_Income             100000 non-null  float64
 5   Monthly_Inhand_Salary     100000 non-null  float64
 6   Num_Bank_Accounts         100000 non-null  float64
 7   Num_Credit_Card           100000 non-null  float64
 8   Interest_Rate             100000 non-null  float64
 9   Num_of_Loan               100000 non-null  float64
 10  Delay_from_due_date       100000 non-null  float64
 11  Num_of_Delayed_Payment    100000 non-null  float64
 12  Changed_Credit_Limit      100000 non-null  float64
 13  Num_Credit_Inquiries      100000 non-null  fl

In [170]:
## 수치형 다음으로 string
## "most_frequent"도 대체해주기.

imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(df[['Payment_Behaviour']])
df[['Payment_Behaviour']] = imputer.transform(df[['Payment_Behaviour']])

# 3. Outliers 처리
지나치게 작거나 많은 Outliers 처리

In [171]:
## replace Outliers with median 
from datasist.structdata import detect_outliers

Numericals_out = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Interest_Rate', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Amount_invested_monthly', 'Monthly_Balance', 'Credit-Builder Loan',
       'Personal Loan', 'Debt Consolidation Loan', 'Student Loan',
       'Payday Loan', 'Mortgage Loan', 'Auto Loan', 'Home Equity Loan']

for col in Numericals_out :
    outliers_indecies = detect_outliers(df,0,[col])
    median = df[col].median()
    df[col].iloc[outliers_indecies] = median

# 4. 범주형데이터 처리

**범주형(str)데이터 찾아서 수치로 바꾸어주기 - getdummies**

In [172]:
df.select_dtypes(include="object")

,Month,Occupation,Payment_of_Min_Amount,Payment_Behaviour,Credit_Score
0,January,Scientist,No,High_spent_Small_value_payments,Good
1,February,Scientist,No,Low_spent_Large_value_payments,Good
2,March,Scientist,No,Low_spent_Medium_value_payments,Good
3,April,Scientist,No,Low_spent_Small_value_payments,Good
4,May,Scientist,No,High_spent_Medium_value_payments,Good
...,...,...,...,...,...
99995,April,Mechanic,No,High_spent_Large_value_payments,Poor
99996,May,Mechanic,No,High_spent_Medium_value_payments,Poor
99997,June,Mechanic,No,High_spent_Large_value_payments,Poor
99998,July,Mechanic,No,Low_spent_Large_value_payments,Standard


"Credit_Score" 는 값 종류가 3개라 열3개로 만들어주지 말고 0,1,2로 대체

In [173]:
df['Credit_Score'].value_counts()

Standard    53174
Poor        28998
Good        17828
Name: Credit_Score, dtype: int64

In [174]:
m = {
    "Poor":0,
    "Standard":1,
    "Good":2
}

df['Credit_Score'] = df['Credit_Score'].map(m)

나머지 범주형 행에 대해서는 getdummies로 한 번에 바꾸기

In [175]:
df = pd.get_dummies(df,drop_first=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 54 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Age                                                 100000 non-null  float64
 1   Annual_Income                                       100000 non-null  float64
 2   Monthly_Inhand_Salary                               100000 non-null  float64
 3   Num_Bank_Accounts                                   100000 non-null  float64
 4   Num_Credit_Card                                     100000 non-null  float64
 5   Interest_Rate                                       100000 non-null  float64
 6   Num_of_Loan                                         100000 non-null  float64
 7   Delay_from_due_date                                 100000 non-null  float64
 8   Num_of_Delayed_Payment                              100000 non-nu

# 5. 데이터 확인 및 저장

In [ ]:
df.describe()

In [176]:
df.to_csv("./csv/Preprocessed_Data.csv") #로 csv추출 가능

전처리 끝!
---

---